In [1]:
import pandas as pd
import pandas_datareader as pdr
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.analyzers as btanalyzers
#import eikon as ek
#matplotlib 3.2.2


from DataManager import DataManager
from SeriesRules import SeriesRules
from ThreeTwelveCross import ThreeTwelveCross
from Report import Report
from MoneyCreditGrowth import MoneyCreditGrowth
from IndicatorStrat import IndicatorStrat

In [2]:
#download the data and stock them in the csv file
start = datetime.datetime (1900, 5, 1)
end = datetime.datetime (2040, 6, 1)
dm = DataManager(start, end)
dm.create_csv()

In [3]:
#dm.showplots(['INDPRO','MYAGM1USM052S'])

In [4]:
#dm.displayDF(['INDPRO','MYAGM1USM052S'])

In [5]:
#dm.setstartdate(datetime.datetime (1999, 5, 1))

In [6]:
#dm.displayDF(['INDPRO','MYAGM1USM052S'])

In [7]:
#dm.setstartdate(datetime.datetime (1950, 1, 1))
#dm.displayDF(['MYAGM1USM052S'])
#dm.showplots(['MYAGM1USM052S'])

In [8]:
 df = pd.read_csv('Data/MYAGM1USM052S.csv')
df["MYAGM1USM052S"][1]

139400000000.0

In [ ]:
#create the three month rule
sr = SeriesRules(dm.getlabeldict().keys())
listRule = sr.createthreemonthrule()
dm.addcolumn_csv(dm.getlabeldict().keys(),"3m growth rate", listRule)
#dm.getlabeldict().keys()

C:\Users\phamk\Documents\GitHub\MacroEconomy\SeriesRules.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  growth = (((df[key][i] /df[key][i-3])**(12/3))-1)*100


In [ ]:
#dm.getdfdict()["IC4WSA"].head(50)

In [ ]:
#create the twelve month rule
listRule = sr.createtwelvemonthrule()
#print(listRule["IC4WSA"])
dm.addcolumn_csv(dm.getlabeldict().keys(),"12m growth rate", listRule)

In [ ]:
# start = datetime.datetime (1900, 5, 1)
# end = datetime.datetime (2040, 6, 1)
# dm = DataManager(start, end)

# Generate the report base on the data mention just before
rep=Report()
for key, value in dm.getmoneycreditgrowth().items():
    res = rep.generateStratChart(key,value)
    rep.createCsvReport(res, key,value)
mc = MoneyCreditGrowth()
mc.computeSumIndex()

In [ ]:
# # class MyBuySell(bt.observers.BuySell):
# #     plotlines = dict(
# #         buy=dict(marker='$\u21E7$', markersize=6.0),
# #         sell=dict(marker='$\u21E9$', markersize=6.0)
# #     )    
    
# cerebro = bt.Cerebro()

# # better net liquidation value view
# cerebro.addobserver(bt.observers.Value,barplot=True,
#         bardist=0.1)

# bt.observers.BuySell = MyBuySell

#
#This cell code visualise the money credit growth module using five indicators. 
#
cerebro = bt.Cerebro()

cerebro = bt.Cerebro(stdstats=False)  # remove the standard observers
cerebro.addobserver(bt.observers.Trades)

cerebro.addobserver(
        bt.observers.BuySell,
        barplot=True,
        bardist=0.015)

cerebro.addstrategy(IndicatorStrat)


data1 = btfeeds.GenericCSVData(
    dataname='Data/TB3MS.csv',

    fromdate=datetime.datetime(1985, 1, 4),
    todate=datetime.datetime(2010, 1, 4),
    timeframe=bt.TimeFrame.Months,
    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),

    datetime=0,
    high=1,
    low=1,
    open=1,
    close=1,
    volume=-1,
    openinterest=-1
) 
cerebro.adddata(data1, name='3-month-yield')

data2 = btfeeds.GenericCSVData(
    dataname='DataReport/results-MoneyCredit.csv',

    fromdate=datetime.datetime(1985, 1, 4),
    todate=datetime.datetime(2010, 1, 4),
    timeframe=bt.TimeFrame.Months,
    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),

    datetime=1,
    high=8,
    low=8,
    open=8,
    close=8,
    volume=-1,
    openinterest=-1
) 

cerebro.adddata(data2, name='Money Credit')

# data3 = btfeeds.GenericCSVData(
#     dataname='data/MYAGM1USM052S.csv',

#     fromdate=datetime.datetime(1960, 1, 4),
#     #     todate=datetime.datetime(1984, 12, 31),
#     todate=datetime.datetime(1969, 1, 4),
#     timeframe=bt.TimeFrame.Months,
#     nullvalue=0.0,

#     dtformat=('%Y-%m-%d'),

#     datetime=0,
#     high=3,
#     low=3,
#     open=3,
#     close=3,
#     volume=-1,
#     openinterest=-1
# )

# data3.compensate(data2)  # let the system know ops on data1 affect data0
# data3.plotinfo.plotmaster = data2
# data3.plotinfo.sameaxis = True

# cerebro.adddata(data3, name='m1 12 month growth')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='analyserTB3')

thestrats = cerebro.run()
thestrat = thestrats[0]

cerebro.plot()

result = thestrat.analyzers.analyserTB3.get_analysis()

listNames = ["title","total","win-count",
        "percent-win","pnl-average", "trade-len","expected-value"]
dictDf={}
dictDf[listNames[0]] = "Indicator over TB3"
dictDf[listNames[1]] = result.total.total
dictDf[listNames[2]] = result.won.total
dictDf[listNames[3]] = float(dictDf["win-count"])/float(dictDf["total"])
dictDf[listNames[4]] = result.pnl.gross.average
dictDf[listNames[5]] = result.len.average
dictDf[listNames[6]] = dictDf[listNames[3]] * dictDf[listNames[4]]
df = pd.DataFrame(dictDf.items(), columns=["label","value"])
        #title
print(df.loc[ 0 , : ]["value"])
display(df)
# lol =  thestrat.analyzers.mytradeanalyzer.get_analysis() 
# print("Total trades Open/Close")
# print('TradeAnalyzer:', lol.total)
# print("----------------------")
# print("PNL")
# print('TradeAnalyzer:', lol.pnl)
# print("----------------------")
# #print('TradeAnalyzer:', lol.streak)
# print("Win count")
# print('TradeAnalyzer:', lol.won)
# print("----------------------")
# print("Lost count")
# print('TradeAnalyzer:', lol.lost)
# print("----------------------")
# print("Trade Lenght")
# print('TradeAnalyzer:', lol.len)
# #print('TradeAnalyzer:', lol.long)
# #print('TradeAnalyzer:', lol.short)
# #df = pd.DataFrame(lol, columns=lol.keys())

In [ ]:
# ek.set_app_key('58cfa4c91e5a4d74bd5d965ba676c3480f95b6fd')
# df, err = ek.get_data(['GOOG.O','MSFT.O', 'FB.O'], 

#                       [ 'TR.Revenue','TR.GrossProfit'])

# df